In [1]:
import pandas as pd
import geopandas as gp
import folium
import numpy as np

In [20]:
year = 2018

In [21]:
acs = pd.read_csv('C:/Workspace/TNC-Demand-Model/Outputs/ACS_' + str(year) + '.csv')

In [22]:
tracts = gp.read_file('C:/Workspace/TNC-Demand-Model/Inputs/Census Shapefiles/Chicago Tracts/geo_export_558aad9f-98d8-4dd5-a6b1-c1730155d596.shp')

In [23]:
tracts.geoid10

0      17031842400
1      17031840300
2      17031841100
3      17031841200
4      17031839000
          ...     
796    17031070400
797    17031070500
798    17031130300
799    17031292200
800    17031630900
Name: geoid10, Length: 801, dtype: object

In [24]:
acs['CENSUS_TRACT'] = acs.CENSUS_TRACT.astype(str)

In [25]:
geo_area = gp.read_file('C:/Workspace/TNC-Demand-Model/Inputs/Census Shapefiles/Tracts/tl_2019_17_tract/tl_2019_17_tract.shp')

geo_area['SQUARE_MILES_LAND'] = geo_area['ALAND']*3.86102e-7
geo_area['GEOID'] = geo_area.GEOID.astype(str)

acs = acs.merge(geo_area[['GEOID','SQUARE_MILES_LAND']], how = 'left', left_on = 'CENSUS_TRACT', right_on = 'GEOID')

In [26]:
acs['TOTAL_POP_PER_SQ_MILE'] = acs['TOTAL_POP'] / acs['SQUARE_MILES_LAND']
acs['TOTAL_HHLDS_PER_SQ_MILE'] = acs['TOTAL_HHLDS'] / acs['SQUARE_MILES_LAND']

In [27]:
normalize_vars = ['AGE_25_34', 'TOTAL_POP_25P_BACHELORS_HIGHER','TOTAL_HHLDS_200KP' ]

for var in normalize_vars:
    acs[var] = acs[var].replace('-',np.nan)
    acs[var] = acs[var].replace('250,000+','250000')
    acs[var] = acs[var].astype(float)
    acs[var + '_PER_POP'] = acs[var]/acs['TOTAL_POP']

In [28]:
centroids = pd.read_csv('C:/Workspace/TNC-Demand-Model/otp/points.csv')

centroids = gp.GeoDataFrame(centroids)

In [29]:
acs['HHLDS_0_VEH_PER_HHLD'] = acs['HHLDS_0_VEH']/acs['TOTAL_HHLDS']

In [30]:
acs['TOTAL_HHLDS_100K_150K'] = acs['TOTAL_HHLDS_100K_150K'].replace('-',np.nan)
acs['TOTAL_HHLDS_150K_200K'] = acs['TOTAL_HHLDS_150K_200K'].replace('-',np.nan)
acs['TOTAL_HHLDS_200KP'] = acs['TOTAL_HHLDS_200KP'].replace('-',np.nan)

acs['TOTAL_HHLDS_100K_150K'] = acs['TOTAL_HHLDS_100K_150K'].astype(float)
acs['TOTAL_HHLDS_150K_200K'] = acs['TOTAL_HHLDS_150K_200K'].astype(float)
acs['TOTAL_HHLDS_200KP'] = acs['TOTAL_HHLDS_200KP'].astype(float)


acs['HHLDS_100KP_PER_HHLD'] = (acs['TOTAL_HHLDS_100K_150K'] +acs['TOTAL_HHLDS_150K_200K'] + acs['TOTAL_HHLDS_200KP'])/acs['TOTAL_HHLDS'] 

In [31]:
variables = acs.columns.drop(['Unnamed: 0', 'CENSUS_TRACT', 'GEOID'])

In [32]:
acs['TOTAL_HHLDS_100K_150K'] = acs['TOTAL_HHLDS_100K_150K'].astype(float)
acs['TOTAL_HHLDS_150K_200K'] = acs['TOTAL_HHLDS_150K_200K'].astype(float)
acs['TOTAL_HHLDS_200KP'] = acs['TOTAL_HHLDS_200KP'].astype(float)


In [33]:
acs[acs['SQUARE_MILES_LAND'] > 10].GEOID.astype(str)

780     17031804201
1081    17031823801
Name: GEOID, dtype: object

In [34]:
for var in variables:
    acs[var] = acs[var].replace('-',np.nan)
    acs[var] = acs[var].replace('250,000+','250000')

    acs[var] = acs[var].astype(float)
    print('Working on variable ' + var)
    m = folium.Map([41.8781, -87.6298], zoom_start=11, tiles = 'stamen toner')
    print(acs[var].max())
    print(acs[var].min())
    print(acs[var].mean())

    bins = np.nanquantile(acs[var], [0, 0.5, 0.75,0.98,1])
    print(bins)
    
    # Add the color for the chloropleth:
    m.choropleth(
     geo_data=tracts,
     name= var,
     data=acs,
     columns = ['CENSUS_TRACT', var],
     key_on='feature.properties.geoid10',
     fill_color='BuGn',
     fill_opacity=0.6,
     line_opacity=0.2,
     bins = bins,
     legend_name= var + ' Totals',
     highlight = True
    )

    feature_group = folium.FeatureGroup(name='Census Tract Centroids', show = False)

    for tract2 in centroids.GEOID:
        row = centroids[centroids['GEOID'] == tract2]
        folium.CircleMarker([row['Y'], row['X']], popup = str(int(row['GEOID'].values[0])), radius = 1, fill = True, fill_color = 'grey', color = 'grey').add_to(feature_group)
    
    m.add_child(feature_group)
    
    folium.LayerControl().add_to(m)

    m.save('C:/Workspace/TNC-Demand-Model/Data Exploration/ACS Maps/' + str(year) + '/' + var + '_Map.html')


Working on variable TOTAL_POP
19889.0
0.0
3960.363153904473
[    0.  3836.  5188.  7832. 19889.]


C:\Users\ramu225\Anaconda3\lib\site-packages\folium\folium.py:415: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


Working on variable TOTAL_MALE
9249.0
0.0
1921.6376042456407
[   0.   1863.   2502.5  3900.88 9249.  ]


C:\Users\ramu225\Anaconda3\lib\site-packages\folium\folium.py:415: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


Working on variable TOTAL_FEMALE
10649.0
0.0
2038.7255496588325
[    0.    1986.    2666.    3988.48 10649.  ]


...

C:\Users\ramu225\Anaconda3\lib\site-packages\folium\folium.py:415: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


Working on variable HHLDS_0_VEH_PER_HHLD
0.7781818181818182
0.0
0.19016950444634645
[0.         0.14546685 0.28091472 0.55177689 0.77818182]


C:\Users\ramu225\Anaconda3\lib\site-packages\folium\folium.py:415: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


Working on variable HHLDS_100KP_PER_HHLD
0.43412698412698414
0.0
0.022358373575894687
[0.         0.01660926 0.02767054 0.08305556 0.43412698]


C:\Users\ramu225\Anaconda3\lib\site-packages\folium\folium.py:415: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning
